# ODA data by Donor and Recipient
This tutorial shows you how to build a Pandas DataFrame containing ODA data to any number of recipients from any number of donors, over a range of years and multiple different indicators.

The tutorial has the following steps:
1. Import `ODAData`, the main tool used to interact with the data
2. Create an instance of `ODAData` with the specific arguments you would like to use
3. Load the indicators and get the DataFrame
4. Optionally, export the DataFrame

## 1. Import ODAData

We can gather the data we need using the `ODAData` class. An object from this class can:
    - Get data for specific indicators
    - Optionally, filter the data for specific donors, recipients and years
    - Optionally, exchange and deflate data

The `oda_data` package automatically downloads the relevant datasets (i.e. DAC1, DAC2a or CRS tables) depending on the indicators selected in step 3.

It is highly recommended that you specify the folder where you want to store, and from where you want to read, this raw data. This must be done once in each notebook or script where we use the `oda_data` package. You can specify the data path if the raw data has already been downloaded, or if you haven't yet specified the download path in your notebook and script.

In [2]:
from oda_data import ODAData, set_data_path

# If you haven't set the data path, you can do it now.
set_data_path(path="../tutorials/data")

# 2. Create an instance of ODAData

Next, we need to set the right arguments which are used to create an instance of ODAData to produce the right DataFrame. The arguments required are:
- *years*: you must specify the `years`, as an `int`, `list` or `range`
- *donors*: you can _optionally_ specify the `donors` you want the output to have (as `int`, or `list`) of donor codes.
- *recipients*: you can _optionally_ specify the `recipients` you want the output to have. Not all indicators
    need or accept recipients. If using an indicator for which recipients are not an option, a warning will be logged to
    the console and the recipients ignored for that indicator.
- *currency*: you can _optionally_ specify the `currency` in which you want your data to be shown. If not specified,
    by default, `USD` will be used. Other options include `EUR`, `GBP` and `CAN`.
- *prices*: you can _optionally_ specify the `prices` in which you want your data to be shown. If not specified,
    by default, `current` will be used. The other option is `constant`. If specifying `constant` a `base_year` must be set.
- *base_year*: you must specify a `base_year` if you have set `prices = 'constant'`. If you have chosen `current` prices,
    by default, `base_year` will be `None`.

You can use a few methods provided by the ODAData object to see the available donors and recipients.

For example:

In [3]:
# print a list of available donors
ODAData().available_donors()

INFO 2023-02-06 12:19:32,715 [oda_data.py:available_donors:445] Note that not all donors may be available for all indicators


{
1: Austria,
2: Belgium,
3: Denmark,
4: France,
5: Germany,
6: Italy,
7: Netherlands,
8: Norway,
9: Portugal,
10: Sweden,
11: Switzerland,
12: United Kingdom,
18: Finland,
20: Iceland,
21: Ireland,
22: Luxembourg,
40: Greece,
50: Spain,
61: Slovenia,
68: Czech Republic,
69: Slovak Republic,
75: Hungary,
76: Poland,
301: Canada,
302: United States,
701: Japan,
742: Korea,
801: Australia,
820: New Zealand,
30: Cyprus,
45: Malta,
55: Turkey,
62: Croatia,
70: Liechtenstein,
72: Bulgaria,
77: Romania,
82: Estonia,
83: Latvia,
84: Lithuania,
87: Russia,
130: Algeria,
133: Libya,
358: Mexico,
543: Iraq,
546: Israel,
552: Kuwait,
561: Qatar,
566: Saudi Arabia,
576: United Arab Emirates,
611: Azerbaijan,
613: Kazakhstan,
732: Chinese Taipei,
764: Thailand,
765: Timor-Leste,
104: Nordic Development Fund,
807: UNEP,
811: Global Environment Facility,
812: Montreal Protocol,
901: International Bank for Reconstruction and Development,
902: Multilateral Investment Guarantee Agency,
903: Internationa

In [4]:
# print a list of available recipients
ODAData().available_recipients()

INFO 2023-02-06 12:19:34,755 [oda_data.py:available_recipients:451] Note that not all recipients may be available for all indicators


{
30: Cyprus,
35: Gibraltar,
45: Malta,
55: Turkey,
57: Kosovo,
61: Slovenia,
62: Croatia,
63: Serbia,
64: Bosnia and Herzegovina,
65: Montenegro,
66: North Macedonia,
71: Albania,
85: Ukraine,
86: Belarus,
88: States Ex-Yugoslavia unspecified,
89: Europe, regional,
93: Moldova,
130: Algeria,
133: Libya,
136: Morocco,
139: Tunisia,
142: Egypt,
189: North of Sahara, regional,
218: South Africa,
225: Angola,
227: Botswana,
228: Burundi,
229: Cameroon,
230: Cabo Verde,
231: Central African Republic,
232: Chad,
233: Comoros,
234: Congo,
235: Democratic Republic of the Congo,
236: Benin,
237: East African Community,
238: Ethiopia,
239: Gabon,
240: Gambia,
241: Ghana,
243: Guinea,
244: Guinea-Bissau,
245: Equatorial Guinea,
247: Cote d'Ivoire,
248: Kenya,
249: Lesotho,
251: Liberia,
252: Madagascar,
253: Malawi,
255: Mali,
256: Mauritania,
257: Mauritius,
258: Mayotte,
259: Mozambique,
260: Niger,
261: Nigeria,
265: Zimbabwe,
266: Rwanda,
268: Sao Tome and Principe,
269: Senegal,
270: Seyche

Next, in order to get the data we want, we will create an instance of the `ODAData` class by specifying the correct arguments.

We will store this instance in a variable called `oda`, which we will use later to load the indicators and get the DataFrame we're after.

Below are some example settings for this tutorial. For clarity, we will first store them in variables, but you can
always pass them directly as arguments to the `ODAData` class.

In [5]:
# Select years as (for example) a range. Remember ranges are exclusive of the upper bound.
years = range(2010, 2021)

# Select donors, which must be specified by their codes. To get all donors, do not use this argument. Here, we are showing France.
donors = 4

# Select recipients, which must be specified by their codes. In this example, we have used Africa, total.
recipients = [10001]

# Select the currency. By default 'USD' is shown but we'll get the data in Euros.
currency = 'EUR'

# Select the prices. By default, 'current' is shown, but we'll get the data in constant prices.
prices = 'constant'

# Set the base year. We must set this given that we've asked for constant data.
base_year = 2021

# Instantiate the `ODAData` class and store it in a variable called 'oda'
oda = ODAData(years=years,
              donors=donors,
              currency=currency,
              prices=prices,
              base_year=base_year,
              include_names=True)

## 3. Load the indicators and get the DataFrame

Then we tell `oda` to load the indicator(s) that are useful for our analysis. For this tutorial, we will use the *"recipient_imputed_multi_flow_net"*, *"recipient_bilateral_flow_net"*, and *"recipient_total_flow_net"* indicators to get total ODA from donor to recipient with a bilateral and multilateral breakdown.

A full list of indicators can be seen by using the `.available_indicators()` method.

In [6]:
# print available indicators
ODAData().available_indicators()

[
total_oda_flow_net,
total_oda_ge,
total_oda_official_definition,
total_oda_bilateral_flow_net,
total_oda_bilateral_ge,
total_oda_multilateral_flow_net,
total_oda_multilateral_ge,
total_oda_flow_gross,
total_oda_flow_commitments,
total_oda_grants_flow,
total_oda_grants_ge,
total_oda_non_grants_flow,
total_oda_non_grants_ge,
total_covid_oda_ge,
total_covid_oda_flow,
total_covid_oda_ge_linked,
total_health_covid_oda_ge,
total_health_covid_oda_flow,
total_health_covid_oda_ge_linked,
total_covid_vaccine_donations_oda_ge,
total_covid_vaccine_donations_oda_flow,
total_covid_vaccine_donations_oda_ge_linked,
total_covid_vaccine_donations_domestic_supply_oda_ge,
total_covid_vaccine_donations_domestic_supply_oda_flow,
total_covid_vaccine_donations_domestic_supply_oda_ge_linked,
total_covid_vaccine_donations_dev_purchase_oda_ge,
total_covid_vaccine_donations_dev_purchase_oda_flow,
total_covid_vaccine_donations_dev_purchase_oda_ge_linked,
total_covid_ancillary_oda_ge,
total_covid_ancillary_oda_fl

In [7]:
# Create a variable with the list of indicators for this analysis.
# The indicator can also be directly passed as an argument in the step below.
indicators = ['recipient_imputed_multi_flow_net','recipient_bilateral_flow_net','recipient_total_flow_net']

# Add all the indicators in our `indicators` list
for indicator in indicators:
    oda.load_indicator(indicator)

df = oda.get_data('all')

# show the resulting dataframe
df

INFO 2023-02-06 12:20:47,920 [read.py:read_dac2a:61] DAC2a data not found. Downloading...
INFO 2023-02-06 12:20:47,921 [dac2a.py:download_dac2a:14] Downloading DAC2A data... This may take a while.
INFO 2023-02-06 12:23:00,510 [common.py:download_single_table:188] Table2a_Data.csv data downloaded and saved.


Data not found, downloading...
Successfully downloaded DAC1 data


,year,indicator,donor_code,donor_name,recipient_code,recipient_name,currency,prices,value
0,2010,recipient_bilateral_flow_net,4,France,55,Turkey,EUR,constant,74.584459
1,2010,recipient_bilateral_flow_net,4,France,57,Kosovo,EUR,constant,1.459793
2,2010,recipient_bilateral_flow_net,4,France,62,Croatia,EUR,constant,3.181168
3,2010,recipient_bilateral_flow_net,4,France,63,Serbia,EUR,constant,11.771164
4,2010,recipient_bilateral_flow_net,4,France,64,Bosnia and Herzegovina,EUR,constant,2.953339
...,...,...,...,...,...,...,...,...,...
7211,2020,recipient_total_flow_net,4,France,10150,"All Multilateral Recipients, Total",EUR,constant,4722.665283
7212,2020,recipient_total_flow_net,4,France,10200,"All Recipients, Total",EUR,constant,18559.187704
7213,2020,recipient_total_flow_net,4,France,10201,"Land-Locked countries, Total",EUR,constant,1986.037917
7214,2020,recipient_total_flow_net,4,France,10202,"Small island states, Total",EUR,constant,719.353569


## 4. Optionally export DataFrame as CSV

Finally, we can export the DataFrame as a CSV if required.

In [13]:
df.to_csv(r'../tutorials/output/total_donor_by_recipient.csv', index = False)